# Coffee Farm Condition Monitoring

This notebook provides code to retrieve data collected using sensors deployed at the Dedan Kimathi University of Technology Coffee farm and display it for visual inspection.

We have deployed the following sensors
1. Ambient Temperature
1. Relative Humidity
1. Soil Temperature 
1. Soil Moisture

The aim is to use these data to help plan farming activities such as irrigation and fungicide application.

The sensors are connected to a [Nucleo F446re](https://os.mbed.com/platforms/ST-Nucleo-F446RE/) development board running code contained in this [repo](https://github.com/ciiram/nyeri-coffee).

In [ ]:
%matplotlib inline
from elasticsearch import Elasticsearch
from elasticsearch import helpers
from pandas import DataFrame, Series
import pandas as pd
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize
from datetime import datetime, timedelta

eshost = "https://search-es-test-ign6g2o7l66nraz2n4gl2pj4dq.eu-west-2.es.amazonaws.com"

es = Elasticsearch(
    [eshost],
    scheme="https",
    port=443,
)

# call ES to return all docs, using size to define the return size of documents
res = es.search(index="coffee_sensors_v2", doc_type="sensor", body={"query": {"match_all": {}}}, size = 2000)
# How many documents
print("%d documents found" % res['hits']['total'])

# read data to data frame
df = json_normalize(res['hits']['hits'])

In [ ]:
df

In [ ]:
df.dtypes

In [ ]:
# convert data type of time to datetime
df[['_source.time']] = df[['_source.time']].apply(pd.to_datetime)

In [ ]:
df.dtypes

In [ ]:
# we are interested in time, ambient temperature and humidity, and soil temperature and humidity
df_coffee_farm = df[['_source.time','_source.dev_id','_source.temperature_2', '_source.relative_humidity_3', '_source.analog_in_4', '_source.analog_in_5']]
df_coffee_farm.columns = ['time','dev_id','Ambient Temperature', 'Relative Humidity', 'Soil Temperature', 'Soil Moisture']

# sorting based on time
df_coffee_farm.sort_values(by='time');

## Ambient and Soil Temperature

In [ ]:
plt.plot(df_coffee_farm.time + timedelta(hours=3), df_coffee_farm['Ambient Temperature'],'bo')
plt.plot(df_coffee_farm.time + timedelta(hours=3), df_coffee_farm['Soil Temperature'],'go')
plt.legend(['Ambient Temperature', 'Soil Temperature']);
plt.xticks(rotation=45);
plt.ylim([0, 50]);
plt.xlabel('Time');
plt.ylabel('Temperature (Celcius)');

## Soil Moisture

In [ ]:
plt.plot(df_coffee_farm.time + timedelta(hours=3), df_coffee_farm['Soil Moisture'],'bo')
plt.xticks(rotation=45);
plt.ylim([0, 50]);
plt.xlabel('Time');
plt.ylabel('Soil Moisture (%)');

## Relative Humidity

In [ ]:
plt.plot(df_coffee_farm.time + timedelta(hours=3), df_coffee_farm['Relative Humidity'],'bo')
plt.xticks(rotation=45);
plt.ylim([0, 100]);
plt.xlabel('Time');
plt.ylabel('Relative Humidity (%)');